## imports

In [2]:
import pandas as pd
import os
import json

In [3]:
import re
import copy


## READ THE FILE

In [4]:
raw_data = []
for line in open('lighttag_annotations_100.json', 'r', encoding='utf-8'):
    raw_data.append(json.loads(line))
    annotations_and_examples = []
    relations = []
    #у нас джейсон в одну линию записан, поэтому по сути annannotations_and_examples и relations не обнуляются, и накапливают всю
    #инфу, которую надо
    for d in raw_data:
        for k, v in d.items():
            if k == 'annotations_and_examples':
                annotations_and_examples.append(v)
            if k == 'relations':
                relations.append(v)
                
#we have no relations in our task, so we need not them to append, so we need not the_list


In [5]:
#annotations_and_examples contains:

# content
# metadata
# annotations 
# classifications


In [58]:
raw_data = []
for line in annotations_and_examples[0]:
    raw_data.append((line))
    content = []
    metadata = []
    annotations = []
    classifications = []
    for d in raw_data:
        for k, v in d.items():
            if k == 'content':
                content.append(v)
            if k == 'metadata':
                metadata.append(v)
            if k == 'annotations':
                annotations.append(v)
            if k == 'classifications':
                classifications.append(v)

In [59]:
annotations[0][186]

{'start': 1942, 'end': 2003, 'tag': 'п'}

In [60]:
annotations[1][0]

{'start': 348, 'end': 363, 'tag': 'п'}

## READ THE DATA

## TAGS

In [61]:
starts = []
ends = []
tags = []

for i in range(len(annotations)):
    raw_data = []
    for line in annotations[i]:
        raw_data.append((line))
        st = []
        en = []
        tg = []
        for d in raw_data:
            for k, v in d.items():
                if k == 'start':
                    st.append(v)
                if k == 'end':
                    en.append(v)
                if k == 'tag':
                    tg.append(v)
    starts.append(st)
    ends.append(en)
    tags.append(tg)

len(starts)

2

In [62]:
len(starts[1])

247

In [63]:
content[1][starts[1][15]:ends[1][15]]

'Янукович '

###  пустышка, заполняем нулями

In [74]:
tagged_content = copy.copy(content)

In [75]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, len(content)):
    print(len(tagged_content[i])==len(content[i]))

True 

True
True


In [76]:
for j in range(0, len(tagged_content)):
    list_content = list(tagged_content[j])

    for i in range(0, len(list_content)):
        if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
            list_content[i] = 'O'

    tagged_content[j] = "".join(list_content)

In [77]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, len(content)):
    print(len(tagged_content[i])==len(content[i]))

True 

True
True


## Избегаем схлопывания многословных тегов: все, что в промежутке тега, заполняем его индикатором, но если этот промежуток совпал с пробелом в оригинальном контенте - тут тоже пробел

In [78]:
for q in range(0, len(tagged_content)):
    list_content = list(tagged_content[q])

    for i in range(0, len(tags[q])):
        for j in range(starts[q][i], ends[q][i]):
            if content[q][j] == ' ':
                list_content[j] = ' '
            else:
                list_content[j] = tags[q][i]

    tagged_content[q] = "".join(list_content)

In [79]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, len(content)):
    print(len(tagged_content[i])==len(content[i]))

True 

True
True


## хлопаем получившиеся посхедовательности до одного символа

In [80]:
for i in range(0, len(tagged_content)):
    tagged_content[i] = re.sub(r'O+', 'O', tagged_content[i])
    tagged_content[i] = re.sub(r'ж+', 'ж', tagged_content[i])
    tagged_content[i] = re.sub(r'п+', 'п', tagged_content[i])
    tagged_content[i] = re.sub(r'д+', 'д', tagged_content[i])
    tagged_content[i] = re.sub(r'м+', 'м', tagged_content[i])

In [81]:
for i in range(0, len(tagged_content)):
    bull = (len(tagged_content[i].split(' ')) == len(content[i].split(' ')))
    print(bull, '\n')

True 

True 



## TAGS AND TEXT BOTH

## Разделяем на тексты и теги к текстам

In [82]:
list_of_tags = []
sub_list_of_tags = []
for i in range(0, len(tagged_content)):
    sub_list_of_tags.append(tagged_content[i].split('\n############\n'))
list_of_tags = sub_list_of_tags[0]+sub_list_of_tags[1]

In [83]:
list_of_texts = []
sub_list_of_texts = []
for i in range(0, len(content)):
    sub_list_of_texts.append(content[i].split('\n############\n'))
list_of_texts = sub_list_of_texts[0]+sub_list_of_texts[1]

In [84]:
print(len(list_of_texts))
print(len(list_of_tags))
#список со всеми текстами и список со всеми текстами тегов, элемент - один текст строкой или один текст тегов строкой

102
102


In [85]:
list_of_text_tokens = []
for i in range(0, len(list_of_texts)):
    list_of_text_tokens.append([list_of_texts[i].split(' ')])

In [86]:
list_of_tags_tokens = []
for i in range(0, len(list_of_tags)):
    list_of_tags_tokens.append([list_of_tags[i].split(' ')])

In [87]:
# в целом те же самые списки, только теперь элемент - список токенов или список тегов (строки разбили на отдельные элементы)

## Проверка совпадения количество тегов - токенов

In [88]:
for i in range(0, len(list_of_text_tokens)):
    if len(list_of_tags_tokens[i][0])!=len(list_of_text_tokens[i][0]):
        print('idx ', i)
        print(len(list_of_tags_tokens[i][0]))
        print(len(list_of_text_tokens[i][0]))
        print('\n')

## Проверка соответствия тегов токенам

In [89]:
#проверяем по-токеново idx-й текст
idx = 100
for i in range(0, len((list_of_text_tokens[idx][0]))):
    if list_of_tags_tokens[idx][0][i] !='O':
        print('слово ',list_of_text_tokens[idx][0][i])
        print('тег ',list_of_tags_tokens[idx][0][i])
        print('\n')

слово  Александр
тег  п


слово  Хорошавин
тег  п


слово  Хорошавин
тег  п


слово  Александр
тег  п


слово  Хорошавин
тег  п


слово  в
тег  д


слово  2011
тег  д


слово  году
тег  д


слово  Сахалинская
тег  ж


слово  компания
тег  ж


слово  Энергострой
тег  ж


слово  Хорошавин
тег  п


слово  Хорошавин
тег  п




## Переворачиваем в нормальные теги

In [90]:
final_list_of_tags_tokens = []
for i in range(0, len(list_of_tags_tokens)):
    sublist = []
    for j in range(0, len(list_of_tags_tokens[i][0])):
        if list_of_tags_tokens[i][0][j] == 'п':
            sublist.append('OFF')
        elif list_of_tags_tokens[i][0][j] == 'ж':
            sublist.append('SUF')
        elif list_of_tags_tokens[i][0][j] == 'д':
            sublist.append('DAT')
        elif list_of_tags_tokens[i][0][j] == 'м':
            sublist.append('LOC')
        else:
            sublist.append('O')
    final_list_of_tags_tokens.append(sublist)
    

In [91]:
len(final_list_of_tags_tokens)

102

In [92]:
len(final_list_of_tags_tokens[0])

1

In [93]:
len(list_of_tags_tokens[0][0])

1

## Заносим все в один датафрейм, подтягиваем столб с оригинальной датой, сохр csv

In [214]:
final = []
for i in range(0, len(list_of_text_tokens)):
    final.append([list_of_text_tokens[i][0], final_list_of_tags_tokens[i]])

In [215]:
str_final = []
for i in range(1, len(final)):
    str_text = ' '.join(final[i][0])
    str_tags = ' '.join(final[i][1])
    str_final.append([str_tags, str_text])
str_final[0][0]

'O O O O O O O O O O OFF O O O O O O O O DAT DAT DAT DAT DAT O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O OFF O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'

### удаляем номер текста и его айдишник (и внешние кавычки) - ужасные артефакты

In [216]:
len(str_final[0][1].split(" "))

107

In [217]:
len(str_final[0][0].split(" "))

107

In [218]:
len(''.join(str_final[0][1].split(",")[1:-1]).split(' '))

107

In [219]:
for i in range(0, len(str_final)):
    str_final[i][1] = ','.join(str_final[i][1].split(",")[1:-1])

In [220]:
for i in range(0, len(str_final)):
    str_final[i][1] = str_final[i][1].replace('"', '')

## все, датафрейм

In [221]:
df = pd.DataFrame(str_final, columns=['tags', 'texts'])
df.head()

,tags,texts
0,O O O O O O O O O O OFF O O O O O O O O DAT DA...,Федеральное налоговое ведомство США по итогам ...
1,O O O O O O O O O O O O O O O O O O O O O O O ...,Приволжское следственное управление на транспо...
2,O O O O O O O O O OFF OFF OFF O O O O O O O O ...,"Уголовное дело , фигурантом которого был генер..."
3,O O O O O O O O O OFF O O O O O O O O O O O O ...,Московская налоговая инспекция проводит новую ...
4,O O O O O OFF OFF O O O O O LOC O O O O O O O ...,Российское следствие объявило украинского бизн...


In [222]:
df['texts'][0]

'Федеральное налоговое ведомство США по итогам аудита обвинило американскую корпорацию Coca-Cola в неуплате налогов на $3 , 3 млрд с 2007 по 2009 годы , сообщает Bloomberg . Пока ведомство не предложило наложить на гиганта рынка штраф . Компания в свою очередь отвергла обвинения и заявила , что намерена искать справедливости в суде . Мы планируем воспользоваться административными и правовыми средствами , которые необходимы для решения вопроса , — говорится в официальном сообщении Coca-Cola . Пресс-служба корпорации подчеркивает , что в течение почти 30 лет следует одной и той же методологии при определении облагаемой налогами части доходов . ,Компанию Coca-Cola обвинили в неуплате налогов на $3,3 млрд'

In [223]:
df.to_csv('Lighttag_0-100_parsed_data_str.csv', index = False)

### проверяем формат строк и все такое при подгрузке сформированного файла обратно

In [224]:
df1 = pd.read_csv('Lighttag_0-100_parsed_data_str.csv')
df1.head()

,tags,texts
0,O O O O O O O O O O OFF O O O O O O O O DAT DA...,Федеральное налоговое ведомство США по итогам ...
1,O O O O O O O O O O O O O O O O O O O O O O O ...,Приволжское следственное управление на транспо...
2,O O O O O O O O O OFF OFF OFF O O O O O O O O ...,"Уголовное дело , фигурантом которого был генер..."
3,O O O O O O O O O OFF O O O O O O O O O O O O ...,Московская налоговая инспекция проводит новую ...
4,O O O O O OFF OFF O O O O O LOC O O O O O O O ...,Российское следствие объявило украинского бизн...


In [225]:
list_of_tags = []
for i in range(0, len(df1['tags'])):
    list_of_tags.append(df1['tags'][i].split(' '))

list_of_texts = []
for i in range(0, len(df1['texts'])):
    list_of_texts.append(df1['texts'][i].split(' '))


In [226]:
for i in range(0, len(list_of_texts)):
    if len(list_of_tags[i])!=len(list_of_texts[i]):
        print('i: ',i)
        print('diff: ',len(list_of_texts[i])- len(list_of_tags[i]))

## MAKING FILES FOR DEEP PAVLOV TRAIN NER

In [228]:
file = open('train.txt','w', encoding='utf-8') 
for i in range(0, int(0.8*len(list_of_texts))):
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
#        if list_of_texts[i][j] == '.':
    file.write('\n')
file.close()

In [229]:
file = open('test.txt','w', encoding='utf-8') 
for i in range(int(0.8*len(list_of_texts)), int(0.9*len(list_of_texts))):
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
#        if list_of_texts[i][j] == '.':
    file.write('\n')
file.close()

In [230]:
file = open('valid.txt','w', encoding='utf-8') 
for i in range(int(0.9*len(list_of_texts)), len(list_of_texts)):
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
#        if list_of_texts[i][j] == '.':
    file.write('\n')
file.close()